In [1]:
from src.functions import *
from scipy.optimize import curve_fit
from joblib import Parallel, delayed

# Global setting
with open(projectdir+'setting.json', 'r') as f:
    setting = json.load(f)
    _gamma = setting["gamma"]
    _T = setting["T"]
    _t = setting["t"]
    _num_random_models = setting["num_random_models"]\

# Load the JSON file
with open(projectdir+'instances.json', 'r') as f:
    instances = json.load(f)

In [2]:
method = 'ra'
weighted = True

In [3]:
qubits_range = np.array([3, 4, 5, 6, 7, 8, 9])
len_qubits_range = len(qubits_range)

In [4]:
delta_arr = np.zeros((len_qubits_range, _num_random_models))

for n_index, n in tqdm(enumerate(qubits_range)):
    def compute_delta(instance, method):
        m = IsingModel.from_coefficients(n, instance)
        data = load_from_json(f'data/n{n}_cubic_bayesian_optimization.json')
        interpolator_kind = data['interpolator_kind']
        schedule = data['schedule_opt']
        t_max = data['t_max']
        schedule_interpolator = get_schedule_interpolator(schedule, kind=interpolator_kind) # get an interpolating function for the continuos schedule
        proposal_mat = get_proposal_mat_ra(m, schedule_interpolator, t_max, assert_symmetry=False)
        P = get_transition_matrix(m, _T, proposal_mat)
        return get_delta(P)

    delta_arr[n_index] = Parallel(n_jobs=-1)(delayed(compute_delta)(instance, method) for instance in instances[f'{n}'])

0it [00:00, ?it/s]

1it [00:02,  2.00s/it]

2it [00:02,  1.04it/s]

3it [00:02,  1.53it/s]

4it [00:03,  1.50it/s]

5it [00:06,  1.53s/it]

6it [00:24,  7.04s/it]

7it [02:42, 50.09s/it]

7it [02:42, 23.25s/it]

In [5]:
# Take an average over random Ising instances
delta_arr_avg = np.mean(delta_arr, axis=1)
delta_arr_std = np.std(delta_arr, axis=1, ddof=1)

In [6]:
if weighted:
    SEM = delta_arr_std/np.sqrt(_num_random_models)
    log_err = SEM/(delta_arr_avg*np.log(2)) # first order error propagation
    fit_coeffs, cov = np.polyfit(qubits_range, np.log2(delta_arr_avg), deg=1, w=1/log_err, cov=True)
else:
    fit_coeffs, cov = np.polyfit(qubits_range, np.log2(delta_arr_avg), deg=1, cov=True)

fit = 2**(fit_coeffs[0]*qubits_range + fit_coeffs[1])
k = fit_coeffs[0]
k_err = np.sqrt(cov[0,0])

In [7]:
qubit_sweep = {'method': method,
               'T': _T,
               'qubits_range': qubits_range.tolist(),
               'num_random_models': _num_random_models,
               'delta': delta_arr.tolist(),
               'delta_avg': delta_arr_avg.tolist(),
               'delta_std': delta_arr_std.tolist(),
               'fit': fit.tolist(),
               'k': k,
               'k_err': k_err,
               'gamma': _gamma
               }

save_in_json(qubit_sweep, f'data/qubit_sweep_{method}.json')